# loading data and creating methods

In [1]:
import pyreason as pr
from loramon import TrainerBase, TrainerBDIF, do_battle, TrainerLLMFeed, TrainerLLMFeedSC, TrainerLLM
from tools.pick_six import agent_create_team
from itertools import permutations
from sim.sim import dict_to_team_set
import google.generativeai as genai
from collections import Counter
from scipy.stats import binomtest
import os

os.environ["API_KEY"] = open("api_key.txt").read()
genai.configure(api_key=os.environ["API_KEY"])

c:\Users\zahdehv\Documents\GITHUB Media\pokemon-python\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def do_matches(p1: TrainerBase,p2: TrainerBase, debug = False):
    teams = []
    for _ in range(6):
        teams.append(dict_to_team_set(agent_create_team()))
    matches = list(permutations(teams, 2))

    victor = []

    for i, mtch in enumerate(matches):
        print("mtch:", i) #, mtch)
        p1.team = mtch[0]
        p2.team = mtch[1]
        try:
            _, winner = do_battle(p1, p2, debug=debug)

            if winner:
                victor.append(winner.name)
                
        except Exception as e:
            print("battle failed:", e)
    return victor


def show_results(names):

    frequency = Counter(names)
    total_count = len(names)

    percentage = {name: (count / total_count) * 100 for name, count in frequency.items()}

    table = []
    table.append(["Name", "Won", "Percentage (%)"])
    for name in frequency:
        table.append([name, frequency[name], round(percentage[name], 2)])

    for row in table:
        print("{:<10} {:<10} {:<10}".format(*row))


# bdi vs random base

In [6]:
BASELINE_RULES=[
    # reglas para la relevancia de deseos
    pr.Rule('desire_h(A,B) <-0 desire_hh(A,B)', 'desire_h_1'),
    pr.Rule('desire_m(A,B) <-0 desire_h(A,B)', 'desire_h_2'),
    pr.Rule('desire_l(A,B) <-0 desire_m(A,B)', 'desire_h_3'),
    pr.Rule('desire_ll(A,B) <-0 desire_l(A,B)', 'desire_h_4'),
    pr.Rule('desire_m(switch,P) <-0 is_switch(switch), is_foe(P, F), on_field_against(C, F), fainted(C)', 'switch_f'),
    
    # reglas para la seleccion de atacar segun efectividad
    pr.Rule('desire_l(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.2,1.0]', 'move1'),
    pr.Rule('desire_m(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.4,1.0]', 'move2'),
    pr.Rule('desire_hh(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.8,1.0]', 'move3'),

    # reglas para el switcheo a un pokemon con mayor ventaja
    pr.Rule('desire_ll(switch,P) <-0 is_foe(P, F), is_switch(switch), has_move(P, M), move_type(M, MT), pokemon_type(F, FT), effectiveness(MT,FT):[0.2,1.0]', 'switch1'),
    pr.Rule('desire_l(switch,P) <-0 is_foe(P, F), is_switch(switch), has_move(P, M), move_type(M, MT), pokemon_type(F, FT), effectiveness(MT,FT):[0.4,1.0]', 'switch2'),
    pr.Rule('desire_m(switch,P) <-0 is_foe(P, F), is_switch(switch), has_move(P, M), move_type(M, MT), pokemon_type(F, FT), effectiveness(MT,FT):[0.8,1.0]', 'switch3'),
]
LESS_SWITCH_RULES=[
    # reglas para la relevancia de deseos
    pr.Rule('desire_h(A,B) <-0 desire_hh(A,B)', 'desire_h_1'),
    pr.Rule('desire_m(A,B) <-0 desire_h(A,B)', 'desire_h_2'),
    pr.Rule('desire_l(A,B) <-0 desire_m(A,B)', 'desire_h_3'),
    pr.Rule('desire_ll(A,B) <-0 desire_l(A,B)', 'desire_h_4'),
    pr.Rule('desire_ll(switch,P) <-0 is_switch(switch), is_foe(P, F), on_field_against(C, F), fainted(C)', 'switch_f'),
    
    # reglas para la seleccion de atacar segun efectividad
    pr.Rule('desire_ll(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.0,1.0]', 'move1'),
    pr.Rule('desire_l(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.2,1.0]', 'move1'),
    pr.Rule('desire_m(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.4,1.0]', 'move2'),
    pr.Rule('desire_hh(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.8,1.0]', 'move3'),

    # reglas para el switcheo a un pokemon con mayor ventaja
    pr.Rule('desire_ll(switch,P) <-0 is_foe(P, F), is_switch(switch), has_move(P, M), move_type(M, MT), pokemon_type(F, FT), effectiveness(MT,FT):[0.4,1.0]', 'switch2'),
    pr.Rule('desire_l(switch,P) <-0 is_foe(P, F), is_switch(switch), has_move(P, M), move_type(M, MT), pokemon_type(F, FT), effectiveness(MT,FT):[0.8,1.0]', 'switch3'),
]

In [8]:
results_bdi_vs_base = do_matches(TrainerBDIF("BDI", LESS_SWITCH_RULES), TrainerBase("BASE"))

mtch: 0


KeyboardInterrupt: 

In [54]:
show_results(results_bdi_vs_base)

Name       Won        Percentage (%)
BDI        28         93.33     
BASE       2          6.67      


In [65]:
binomtest(results_bdi_vs_base.count("BDI"), len(results_bdi_vs_base), p=0.8, alternative='greater')

BinomTestResult(k=28, n=30, alternative='greater', statistic=0.9333333333333333, pvalue=0.04417898515199711)

# bdi vs llm

In [4]:
inst_old = """-In case any rival pokemon is fainted, still select an action, the rival could send another pokemon, in other case,
if the battle has finished, the combat will end automatically so you wont be prompted to give any further actions.
-The user Will also provide you previous turn decisions and their consequenses can be seen at the next turn state. You are going to account them on your decision so you
avoid repeating mistakes or action taken with panic (like switching back and forth).
-In case there are provided reasoning steps from yourself, you are going to analize them all, chech the most frequent actions, and make a 
choice based on that.
-You will also consider avoid repeating and chaotic switch actions according to previous turns.
When you decide on switch, make sure you won't switch back or make a second switch."""

inst = """
- You will make a step by step reasoning, with all available actions, assigning each action a qualification from 'A'(best possible assignation) to 'F'(worst possible assignation).
- From the turn historial, you will try to retrieve information about what pokemons has the rival that are not in the field, so when a pokemon faints, you can choose to switch or prepare
an attack using that information.
- Avoid switching panic (switching between pokemons again and again, use historial to check wether to that current adversary you switched to a specific pokemon, your reasoning took you to
select it).
- You will reassignate categories based on past turn historial (a switch that ended in a bad outcome would be penalized) [examples below]

examples:
use thunderbolt (A) thunderbolt is very effective against POKEMON, so it would be a good option
switch to pikachu (D) pikachu is effective against rival's pokemon, but current pokemon is equally effective, change doesn't make a difference
switch to riolu (B) riolu is equally effective against rival's pokemon, but current pokemon hp is very low, and riolu is on a good state
"""

In [7]:
bdi = TrainerBDIF("BDI", LESS_SWITCH_RULES)
llm = TrainerLLMFeed("LLM", specific_instructions=inst, num_turns=23, talkative= True)
do_battle(bdi, llm, True)

BATTLE STARTED

BDI vs LLM

BDI pokemons are:
lickilicky
crobat
palkia
granbull
cascoon
eevee



LLM pokemons are:
clawitzer
snubbull
croagunk
bidoof
wishiwashi
lickitung



BDI is thinking...
LLM is thinking...

LLM 's INNER DIALOGUE:
Okay, let's analyze the situation and make a decision.

**Reasoning:**

* **Current Pokemon:** Clawitzer is a strong Water-type, but Lickilicky is Normal-type, making Clawitzer's Water-type moves less effective.
* **Opponent's Pokemon:** Lickilicky is a Normal-type, which means it's weak to Fighting-type moves.
* **Available Actions:**
    * **A - Use Focusblast (Fighting):** This is the best option. Focusblast is a powerful Fighting-type move that will be super effective against Lickilicky. 
    * **B - Switch to Croagunk (Poison/Fighting):** Croagunk is also a Fighting-type and would be super effective against Lickilicky. However, Focusblast is a stronger move.
    * **C - Use U-Turn:** This would allow us to switch out to a more advantageous Pokemon, 

KeyboardInterrupt: Interrupted by user

In [9]:
results_bdi_vs_llm = do_matches(TrainerBDIF("BDI", LESS_SWITCH_RULES), TrainerLLMFeed("LLM", specific_instructions=inst, num_turns=23))

mtch: 0
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota).
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota).


KeyboardInterrupt: 

In [64]:
show_results(results_bdi_vs_llm)

NameError: name 'results_bdi_vs_llm' is not defined

In [ ]:
binomtest(results_bdi_vs_llm.count("BDI"), len(results_bdi_vs_llm), p=0.5, alternative='greater')